# Building NMF Topic Model
components = 30

In [2]:
import pandas as pd 
import streamlit as st 
import altair as alt
from PIL import Image 
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
import warnings
warnings.filterwarnings("ignore")
from corextopic import corextopic as ct
from corextopic import vis_topic as vt

In [3]:
# read in wine data
# drop duplicates and fill na with zeros 
WINE = pd.read_csv('WINENLP.csv')
WINE.drop_duplicates(subset=['description'],inplace=True)
WINE.fillna(0,inplace=True)
shp = WINE.shape
shp[0]
WINES = WINE[['description','country','province','variety','price','title']]
WINES.reset_index(inplace=True,drop=True)

In [4]:

CORPUS1 = WINES['description']


In [5]:

STOP = stopwords.words('english')
STOP += ['notes','wine','drink','region','nan','nose',
        'like','made','shows','parts','style','followed','amounts','mouth','flavors']

In [6]:
# instantiate Tfid vectorizer
TF_IDF = TfidfVectorizer(stop_words=STOP,token_pattern = r'(?u)\b[A-Za-z]+\b')

# fit transform returns Sparse Matrix 
Description_Matrix = TF_IDF.fit_transform(CORPUS1)

# words used
words = TF_IDF.get_feature_names()

Description_Matrix


<119955x29722 sparse matrix of type '<class 'numpy.float64'>'
	with 2693261 stored elements in Compressed Sparse Row format>

In [7]:
def fit(df):
    num_components = int(input("How many topics: "))
    # instantiating Non Negative Matrix Factorization Model
    nmf = NMF(n_components=num_components,random_state=84)
    # fitting model 
    nmf.fit(df)
    return nmf , num_components
nmf = fit(Description_Matrix)

How many topics: 30


In [8]:
Topic_Matrix = nmf[0].transform(Description_Matrix)
Topic_Matrix_df = pd.DataFrame(Topic_Matrix).add_prefix('topic_')
Topics = pd.concat([Topic_Matrix_df, WINES], axis=1)
Topics.dropna(inplace=True)
Topics.reset_index(inplace=True,drop=True)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
Topic_Matrix_Partial = Topic_Matrix[:20000]

# Building DataFrames for Pickling
Use in streamlit app

In [10]:
Cosine_Matrix = cosine_similarity(Topic_Matrix_Partial)

In [11]:
COS_mat = pd.DataFrame(Cosine_Matrix)

In [29]:
df_similarity = pd.DataFrame(COS_mat[0])

In [43]:
df = WINES[['price','description','variety','title']]

In [45]:
Attributes = df[:20000]

In [47]:
Attributes['Similarity'] = df_similarity

In [54]:
Attributes.to_pickle('/Users/jamesgardner/Desktop/Projects/NLP/attributes.pkl')

In [53]:
COS_mat.to_pickle('/Users/jamesgardner/Desktop/Projects/NLP/cosine_mat.pkl')